# Building Energy Profiles clustering 

## Import libraries for analysis

In [1]:
# Built-in libraries
import os
import re
import time
from datetime import datetime
import pytz
from math import log

# NumPy, SciPy and Pandas
import numpy as np
import pandas as pd

# Scikit-Learn
from sklearn.preprocessing import StandardScaler

# Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Workalendar
from workalendar.europe import Switzerland
from workalendar.europe import UnitedKingdom
from workalendar.usa import Colorado
from workalendar.usa import NewYork
from workalendar.usa import California
from workalendar.usa import Arizona
from workalendar.usa import Illinois
from workalendar.asia import Singapore
from workalendar.oceania import WesternAustralia

## A. Read HDF5 Data

In [11]:
import h5py
filename = 'meta/meta.hdf5'
buildings_data = h5py.File(filename, 'r')

In [12]:
list(buildings_data.keys())

['MIT', 'genome', 'genome2', 'ireland', 'pecan', 'utexas', 'washington']

In [13]:
# Get all metadata field names
meta_fields = set()
for dset_name in buildings_data:
    for building_name in buildings_data[dset_name]:
        for meta_field in buildings_data[dset_name][building_name].attrs:
            meta_fields.add(meta_field)
meta_fields = list(meta_fields)
meta_fields.sort()

In [14]:
# Show what metadata fields are present in the dataset
meta_fields

['Climatezone', 'Industry', 'PSU', 'Sqft', 'Subindustry', 'Timezone']

In [19]:
# Only use buildings that contain the compulsory fields specified below
compulsory_fields = ['Industry', 'PSU', 'Sqft', 'Timezone', 'Climatezone']

In [20]:
def get_attr(fields, dict_like):
    return [dict_like[f] if f in dict_like else '' for f in fields]

In [21]:
def check_fields(fields, dict_like):
    for f in fields:
        if f not in dict_like:
            return False
        if pd.isnull(dict_like[f]):
            return False
    return True

In [22]:
datetime_format = "%Y-%m-%d %H:%M:%S"

def convert_ts_from_utc(timestamp, to):
    '''Helper function to convert datetime from UTC to specified timezone'''
    datetime_object = datetime.strptime(timestamp, datetime_format).replace(tzinfo=pytz.utc)
    to_datetime = datetime_object.astimezone(pytz.timezone(to))
    return to_datetime.strftime(datetime_format)

In [27]:
start = time.time()

buildings_dict = {}

for dset_name in buildings_data:
    if dset_name not in buildings_dict:
        buildings_dict[dset_name] = {}

    for building_name in buildings_data[dset_name]:
        buildings_dict[dset_name][building_name] = {}
        if not check_fields(compulsory_fields, buildings_data[dset_name][building_name].attrs):
            continue
        meta_attr = get_attr(meta_fields, buildings_data[dset_name][building_name].attrs)
        for year in buildings_data[dset_name][building_name]:
            data = buildings_data[dset_name][building_name][year][:, :].astype(str)
            # convert to local time for genome data
            if dset_name == 'genome' or dset_name == 'genome2':
                timezone = buildings_data[dset_name][building_name].attrs['Timezone']
                data[:, 0] = [convert_ts_from_utc(x, timezone) for x in data[:, 0]]
            buildings_dict[dset_name][building_name][year] = (meta_attr, data)

print('Time spent: %d' % (time.time() - start))

Time spent: 611


## Create Building Profiles

* Once we read the file, we need to generate the daily profiles (24 hourly data points) for clustering analysis with Z-normalization technique

In [28]:
start = time.time()

data = []

for dset_name in buildings_dict:
    for building_k in buildings_dict[dset_name]:
        for year in buildings_dict[dset_name][building_k]:
            building = buildings_dict[dset_name][building_k][year][1]
            meta_attr = buildings_dict[dset_name][building_k][year][0]

            if (building.shape[0] == 0):
                continue
            cur_ts = building[0, 0]
            cur_date = cur_ts[:10]
            building_profiles = [(cur_date, [])]

            for row in building:
                ts = row[0]
                val = row[1]
                if ts[:10] != cur_date:
                    cur_date = ts[:10]
                    building_profiles.append((cur_date, []))
                building_profiles[-1][1].append(val)

            '''Only use profiles with complete 24 electricity readings'''
            complete_profiles =  list(filter(lambda x: len(x[1]) == 24, building_profiles))
            for profile in complete_profiles:
                data.append([dset_name, building_k, profile[0]] + profile[1] + meta_attr)

print('Time spent generating building profiles: %fs' % (time.time() - start))

Time spent generating building profiles: 320.812536s


In [29]:
# Convert data to NumPy array
np_data = np.array(data)

In [30]:
# Show the number of rows and columns
np_data.shape

(2365925, 33)

In [31]:
pd.DataFrame(np_data).to_csv('np_data.csv', index=False)

In [2]:
np_data = pd.read_csv('np_data.csv')
np_data = np_data.as_matrix()

/Users/yangxiya/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (2,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# Normalize the profiles
scaler = StandardScaler()
norm_data = np.concatenate((np_data[:, :3], scaler.fit_transform(np_data[:, 3:3+24].T).T, np_data[:, 3+24:]), axis=1)

/Users/yangxiya/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Attach meta data and calculate EUI

* Basic metadata is stored in the original dataset
* The EUI is calculated 
* Holiday information is also considered for analyzing clustering result by different day types (e.g., weekday, weekend, holiday)
* Note that a complete dataset (.hdf5) has metadata except for profile variation

In [15]:
combined_profiles = pd.DataFrame(norm_data, columns = ['Dataset', 'Building', 'Date'] + ['0%d:00' % i for i in range(10)] + ['%d:00' % i for i in range(10, 24)] + meta_fields)

In [16]:
### Convert Sqft to Sqm ###
one_sqft_in_sqm = 145161 / 1562500 # source: Wolfram Alpha
combined_profiles['Sqft'] = combined_profiles['Sqft'] * one_sqft_in_sqm
columns = combined_profiles.columns.tolist()
columns[-3] = 'Sqm'
combined_profiles.columns = columns

In [23]:
combined_profiles = combined_profiles.loc[(combined_profiles.Sqm != 0), :].copy()

In [24]:
combined_profiles.reset_index(drop=True, inplace=True)

In [28]:
### calculate EUI ###
'''
Use latest 365 days of data. If data is less than 365 days,
add up electricity for the days we have and multiply (365 / # of days of data available)
'''
start = time.time()

cur_building = np_data[-1,1]
energy_dict = {cur_building: 0}
cur_count = 0

for row in np_data[::-1,:]:
    if row[1] == cur_building and cur_count < 365:
        for i in row[3:3+24]:
            energy_dict[cur_building] += float(i)
        cur_count += 1
        continue
    elif row[1] == cur_building:
        continue
    elif cur_count < 365:
        energy_dict[cur_building] *= (365 / cur_count)
    cur_building = row[1]
    energy_dict[cur_building] = 0
    cur_count = 0

building_sqm = {}
for row in combined_profiles.iterrows():
    building = row[1].loc['Building']
    sqm = row[1].loc['Sqm']
    building_sqm[building] = sqm

EUI = {}
EUI.update(energy_dict)
for k in EUI:
    if k in building_sqm:
        EUI[k] /= building_sqm[k]

EUI_col = []
for i in range(combined_profiles.shape[0]):
    EUI_col.append(EUI[combined_profiles.loc[i, 'Building']])

combined_profiles['EUI'] = EUI_col

print('Time spent calculating EUI: %fs' % (time.time() - start))

Time spent calculating EUI: 370.403654s


In [29]:
### Create date flags ###
start = time.time()
# Get holidays
reference_dict = {
    'Europe/Zurich': Switzerland,
    'Europe/London': UnitedKingdom,
    'America/Denver': Colorado,
    'America/New_York': NewYork,
    'America/Los_Angeles': California,
    'America/Phoenix': Arizona,
    'America/Chicago': Illinois,
    'Asia/Singapore': Singapore,
    'Australia/Perth': WesternAustralia,
}

holidays_save = {}

def get_holidays(timezone, year):
    if timezone in holidays_save and year in holidays_save[timezone]:
        return holidays_save[timezone][year]
    holidays = reference_dict[timezone]().holidays(year)
    str_holidays = set(date_obj.strftime("%Y-%m-%d") for (date_obj, _) in holidays)
    if timezone not in holidays_save:
        holidays_save[timezone] = {}
    holidays_save[timezone][year] = str_holidays
    return str_holidays

def is_holiday(date_str, timezone):
    year = int(date_str[:4])
    return date_str in get_holidays(timezone, year)

holiday_flags = np.apply_along_axis(lambda row: is_holiday(row[0], row[1]), 1, combined_profiles[['Date', 'Timezone']].as_matrix())

# Get weekends
def get_weekday_idx(date, timezone):
    datetime_object = datetime.strptime(date, "%Y-%m-%d").replace(tzinfo=pytz.timezone(timezone))
    return datetime_object.weekday()

weekends_idx = set((5, 6))

def is_weekend(date, timezone):
    return get_weekday_idx(date, timezone) in weekends_idx

def get_day_of_year(date):
    '''Compute day of year based on a 366 days calendar (year 2000)'''
    datetime_object = datetime.strptime(date, "%Y-%m-%d").replace(year=2000)
    return (datetime_object - datetime.strptime("2000-01-01", "%Y-%m-%d")).days + 1

weekend_flags = np.apply_along_axis(lambda row: is_weekend(row[0], row[1]), 1, combined_profiles[['Date', 'Timezone']].as_matrix())
dayofyear = np.apply_along_axis(lambda row: get_day_of_year(row[0]), 1, combined_profiles[['Date']].as_matrix())

# Build date flags
dateflag = np.array(['weekday' for _ in weekend_flags])
dateflag[weekend_flags] = 'weekend'
dateflag[holiday_flags] = 'holiday'

combined_profiles['dateflag'] = dateflag
combined_profiles['dayofyear'] = dayofyear

print('Time spent building date flags and dayofyear values: %fs' % (time.time() - start))

/Users/yangxiya/anaconda3/lib/python3.5/site-packages/workalendar/core.py:644: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


Time spent building date flags and dayofyear values: 135.494549s


In [30]:
Industries = set([
    'Residential',
    'Education',
    'Government'
])

PSUs = set([
    'Single_family_house',
    'Office',
    'College Classroom',              
    'College Laboratory',
    'Primary/Secondary Classroom',
    'Dormitory',
    'Library',
    'Industrial',
    'Gymnasium',
    'Community Center',
    'Food Sales',
    'Sports Stadium',
    'Student Union',
    'Retail',
    'Museum',
    'Fitness Center'
])

industry = combined_profiles.Industry.tolist()
new_industry = []
for ind in industry:
    if ind not in Industries:
        new_industry.append('Others')
    else:
        new_industry.append(ind)

psus = combined_profiles.PSU.tolist()
new_psu = []
for psu in psus:
    if psu not in PSUs:
        new_psu.append('Others')
    else:
        new_psu.append(psu)

In [31]:
combined_profiles['Industry'] = new_industry
combined_profiles['PSU'] = new_psu
combined_profiles.to_csv('final_profiles.csv', index=False)

In [32]:
combined_profiles.head()

,Dataset,Building,Date,00:00,01:00,02:00,03:00,04:00,05:00,06:00,...,23:00,Climatezone,Industry,PSU,Sqm,Subindustry,Timezone,EUI,dateflag,dayofyear
0,pecan,3831,2015-01-01,-0.22758,-1.90817,-0.22758,0.612716,-0.22758,0.612716,1.87316,...,0.612716,2,Residential,Single_family_house,105.909,NaN,America/Chicago,45.627032,holiday,1
1,pecan,3831,2015-01-02,0.479542,-0.835773,0.479542,0.808371,0.479542,-0.835773,0.479542,...,0.808371,2,Residential,Single_family_house,105.909,NaN,America/Chicago,45.627032,weekday,2
2,pecan,3831,2015-01-03,1.45521,0,1.45521,0,1.45521,1.45521,0,...,-0.485071,2,Residential,Single_family_house,105.909,NaN,America/Chicago,45.627032,weekend,3
3,pecan,3831,2015-01-04,-0.3897,-0.471384,-0.3897,-0.471384,-0.430542,-0.471384,-0.430542,...,1.40734,2,Residential,Single_family_house,105.909,NaN,America/Chicago,45.627032,weekend,4
4,pecan,3831,2015-01-05,-0.216109,-0.165753,-0.266464,-0.165753,-0.266464,-0.165753,0.0860239,...,0.438512,2,Residential,Single_family_house,105.909,NaN,America/Chicago,45.627032,weekday,5
